In [ ]:
##### Here is a demonstration of installing Scythe.jl through Jupyter Notebook

# using Pkg
# Pkg.add(path="/path/to/Scythe.jl")

##### If other packages have not been installed yet, user can uncomment the following line

# Pkg.add("Distributed")


In [ ]:
using Distributed
addprocs(2)
@everywhere using Scythe
@everywhere using DistributedData
@everywhere using SharedArrays
@everywhere using SparseArrays
using CSV
using DataFrames

In [ ]:
spinup = "./symmetric_spinup"
if !isdir(spinup)
    mkdir(spinup)
end

In [ ]:
model = ModelParameters(
    ts = 3.0,
    integration_time = 10800.0,
    output_interval = 3600.0,
    equation_set = "Oneway_ShallowWater_Slab",
    initial_conditions = "./symmetric_spinup/SWslab_OnewayRankine.csv",
    output_dir = "./symmetric_spinup/",
    grid_params = Scythe.GridParameters(
        geometry="RL",
        xmin = 0.0,
        xmax = 3.0e5,
        num_cells = 100,
        BCL = Dict(
            "h" => Scythe.CubicBSpline.R1T1,
            "u" => Scythe.CubicBSpline.R1T0,
            "v" => Scythe.CubicBSpline.R1T0,
            "ub" => Scythe.CubicBSpline.R1T0,
            "vb" => Scythe.CubicBSpline.R1T0,
            "wb" => Scythe.CubicBSpline.R1T1),
        BCR = Dict(
            "h" => Scythe.CubicBSpline.R0,
            "u" => Scythe.CubicBSpline.R1T1,
            "v" => Scythe.CubicBSpline.R0,
            "ub" => Scythe.CubicBSpline.R1T1,
            "vb" => Scythe.CubicBSpline.R0,
            "wb" => Scythe.CubicBSpline.R0),
        vars = Dict(
            "h" => 1,
            "u" => 2,
            "v" => 3,
            "ub" => 4,
            "vb" => 5,
            "wb" => 6)),
        physical_params = Dict(
            :g => 9.81,
            :K => 3000.0,
            :Cd => 2.4e-3,
            :Hfree => 2000.0,
            :Hb => 1000.0,
            :f => 5.0e-5))

In [ ]:
scythegrid = Scythe.createGrid(model.grid_params)
gridpoints = Scythe.getGridpoints(scythegrid);

In [ ]:
Rmax = 50000.0
Vmax = 50.0
V0 = Vmax / Rmax
zeta = 2.0 * V0
epsilon = 5000.0
f = 5.0e-5
r_1 = 0.0
h = 0.0
ubar = 0.0
vbar = 0.0
vprime = 0.0
uprime = 0.0
for i = 1:scythegrid.params.lDim
    r_m = gridpoints[i,1]
    l_m = gridpoints[i,2]
    if (r_m < Rmax)
        vbar = V0 * r_m
    else
        vbar = Rmax * Rmax * V0 / r_m
    end
    if r_m > r_1
        dhdr = ((f * vbar) + (vbar * vbar / r_m)) / 9.81
        h = h + (dhdr * (r_m - r_1))
        r_1 = r_m
    end
    scythegrid.physical[i,1,1] = h
    scythegrid.physical[i,2,1] = ubar
    scythegrid.physical[i,3,1] = vbar
    scythegrid.physical[i,4,1] = ubar
    scythegrid.physical[i,5,1] = vbar
    scythegrid.physical[i,6,1] = 0.0
end

In [ ]:
rankine_ics = DataFrame(r=gridpoints[:,1],
    l=gridpoints[:,2], 
    h=scythegrid.physical[:,1,1], 
    u=scythegrid.physical[:,2,1], 
    v=scythegrid.physical[:,3,1],
    ub = scythegrid.physical[:,4,1],
    vb = scythegrid.physical[:,5,1],
    wb = scythegrid.physical[:,6,1]); 

In [ ]:
CSV.write(spinup*"/SWslab_OnewayRankine.csv",rankine_ics)

In [ ]:
# Run the 3-hr spinup
integrate_model(model)

In [ ]:
balanced = CSV.read("./symmetric_spinup/physical_out_10800.0.csv", DataFrame, header=1);

In [ ]:
Rmax = 50000.0
Vmax = 50.0
V0 = Vmax / Rmax
zeta = 2.0 * V0
epsilon = 5000.0
f = 5.0e-5
r_1 = 0.0
h = 0.0
ubar = 0.0
vbar = 0.0
vprime = 0.0
uprime = 0.0
for i = 1:scythegrid.params.lDim
    r_m = gridpoints[i,1]
    l_m = gridpoints[i,2]
    R_eli = Rmax + (epsilon * cos(2 * l_m))
    
    # Use the output field for mean vortex
    vfree = balanced.v[i]
    ufree = balanced.u[i]
    h = balanced.h[i]
    vbl = balanced.vb[i]
    ubl = balanced.ub[i]
    wbl = balanced.wb[i]    

    if (r_m < Rmax)
        vprime = 0.5 * zeta * r_m * (epsilon * cos(2 * l_m) / Rmax)
        uprime = 0.5 * zeta * r_m * (epsilon * sin(2 * l_m) / Rmax)
    else
        vprime = 0.5 * zeta * (Rmax^2 / r_m) * (-epsilon * cos(2 * l_m) * Rmax / r_m^2)
        uprime = 0.5 * zeta * (Rmax^2 / r_m) * (epsilon * sin(2 * l_m) * Rmax / r_m^2)
    end
    scythegrid.physical[i,1,1] = h
    scythegrid.physical[i,2,1] = ufree + uprime
    scythegrid.physical[i,3,1] = vfree + vprime
    scythegrid.physical[i,4,1] = ubl + uprime
    scythegrid.physical[i,5,1] = vbl + vprime
    scythegrid.physical[i,6,1] = wbl
end

In [ ]:
wave2_ics = DataFrame(r=gridpoints[:,1],
    l=gridpoints[:,2], 
    h=scythegrid.physical[:,1,1], 
    u=scythegrid.physical[:,2,1], 
    v=scythegrid.physical[:,3,1],
    ub = scythegrid.physical[:,4,1],
    vb = scythegrid.physical[:,5,1],
    wb = scythegrid.physical[:,6,1]);

In [ ]:
oneway = "./oneway_wave2"
if !isdir(oneway)
    mkdir(oneway)
end

In [ ]:
CSV.write(oneway*"/SWslab_wave2.csv",wave2_ics)

In [ ]:
# To reproduce the one-way model results in Cha and Bell (2024)
# Change the initial conditions and integration time in the model structure to:
#
#    initial_conditions = "./oneway_wave2/SWslab_wave2.csv",
#    integration_time = 86400.0,
#
# and run integrate_model(model) again
#
# To reproduce the two-way model results in Cha and Bell (2024)
# Change the IC and integration as above and change equation set to:
#
#    equation_set = "Twoway_ShallowWater_Slab",
#
# and run integrate_model(model) again
#
# Running in a terminal or job submission instead of the notebook is recommended
# for the full 24-hour simulations